In [1]:
import numpy as np
import pandas as pd
from collections import Counter
import csv
import string
from sklearn.dummy import DummyClassifier

from sklearn.svm import LinearSVC
from collections import Counter
import math
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
from scipy.sparse import hstack
from sklearn.naive_bayes import BernoulliNB
import scipy.sparse as sps
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score 
from sklearn.dummy import DummyClassifier

In [2]:
table = {ord(char): None for char in string.punctuation}
R = open("Datasets/IMDB-train.txt","r").read().translate(table).lower()
Text = R.split(" ")
imdb_top = Counter(Text).most_common(10000)
dic_y = dict(imdb_top)
A = {}
i = 0
for k in dic_y:
    A[k] = i
    i+= 1

yelp_v = open("imdb-vocab.txt","a")
for i in range(10000):
	line = str(i+1) + "\t" + imdb_top[i][0] + "\t" + str(imdb_top[i][1]) + "\n"
	yelp_v.write(line)


In [3]:
imdb_train_x = []
imdb_train_y = []
f = open("Datasets/IMDB-train.txt","r").read().translate(table).lower()
x = f.split("\n")
c = 0
for i in range(len(x)-1):
    a = x[i].split("\t")
    imdb_train_x = imdb_train_x + [a[0]]
    imdb_train_y = imdb_train_y + [int(a[1])]
                        
imdb_valid_x = []
imdb_valid_y = []
f = open("Datasets/IMDB-valid.txt","r").read().translate(table).lower()
x = f.split("\n")
for i in range(len(x)-1):
    a = x[i].split("\t")
    imdb_valid_x = imdb_valid_x + [a[0]]
    imdb_valid_y = imdb_valid_y + [int(a[1])]
                        
imdb_test_x = []
imdb_test_y = []
f = open("Datasets/IMDB-test.txt","r").read().translate(table).lower()
x = f.split("\n")
for i in range(len(x)-1):
    a = x[i].split("\t")
    imdb_test_x = imdb_test_x + [a[0]]
    imdb_test_y = imdb_test_y + [int(a[1])]

In [4]:
def bagofwords(data, dic,w):
    DM_binary = []
    DM_frequency = []
    for x in data:
        vector_binary = np.zeros(len(dic))
        vector_frequency = np.zeros(len(dic))
        for y in x:
            if y in dic:
                index = w[y]
                if (vector_binary[index] == 0):
                    vector_binary[index] = 1
                vector_frequency[index] += 1                
        DM_binary.append(vector_binary)
        DM_frequency.append(vector_frequency)   
    return DM_frequency, DM_binary




In [ ]:
train_i_f, train_i_b = bagofwords(imdb_train_x, dic_y,A)

In [ ]:
valid_i_f, valid_i_b = bagofwords(imdb_valid_x, dic_y,A)

In [ ]:
test_i_f, test_i_b = bagofwords(imdb_test_x, dic_y,A)

In [ ]:
def performance(train_y_b,yelp_train_y,valid_y_b,yelp_valid_y,test_y_b,yelp_test_y):
    uniform= DummyClassifier(strategy='uniform')
    uniform.fit(train_y_b, yelp_train_y)
    print("The f1_measure on test set for Random classifier is :" + str(f1_score(yelp_test_y,uniform.predict(test_y_b), average = 'micro')))


    majority_classifier = DummyClassifier(strategy='most_frequent') 
    majority_classifier.fit(train_y_b, yelp_train_y)

    print("The f1_measure on test set for majority-class classifier is :" + str(f1_score(yelp_test_y, majority_classifier.predict(test_y_b), average = 'micro')))
    
    
    classifier = BernoulliNB()
    classifier.fit(train_y_b, yelp_train_y)
    parameters = []
    for i in range(0,100):
        parameters.append(math.pow(10,(i-90)))
    best = -1
    k = 0
    for i in parameters:
        classifier.alpha = i
        f1_measure = f1_score(yelp_valid_y,classifier.predict(valid_y_b), average='micro')
        if (best < f1_measure):
            k = i
            best = f1_measure
    classifier.alpha = k   

    pn_test = classifier.predict(test_y_b)

    test= f1_score(yelp_test_y,pn_test, average='micro')
    print()
    print("Test Result - naive classifier:")
    print("The F-measure is " + str(test) + "with best alpha is:" +str(classifier.alpha) ) 
    
    max_features = [i for i in range(1,20)]
    min_samples_leaf = [i for i in range(1,20)]
    max_depth = [i for i in range(1,20)]
    tre = tree.DecisionTreeClassifier()
    tre.fit(train_y_b, yelp_train_y)
    best_f1_2 = -10
    best_i = 0
    best_j = 0
    best_k = 0 
    for i in max_features:
        for j in min_samples_leaf:
            for k in max_depth:
                tree.max_features = i
                tree.min_samples_leaf = j
                tree.max_depth = k
                f1_measure = f1_score(yelp_valid_y,tre.predict(valid_y_b), average='micro')
                if (best_f1_2 < f1_measure):
                    best_i = i
                    best_j = j
                    best_k = k 
                    best_f1_2 = f1_measure
                

    tre.max_features = best_i
    tre.max_features = best_j
    tre.max_features = best_k 


    pt_test = tre.predict(test_y_b)
    test = f1_score(yelp_test_y,pt_test, average='micro')
    print()
    print("Test Result - tree classifier:")
    print("The F-measure is " + str(test)) 
    
    linear= LinearSVC()
    linear.fit(train_y_b, yelp_train_y)
    #svc_valid = lin_clf.predict(valid_y_b)
    parameters = []
    for i in range(0,100):
        parameters.append(math.pow(10,(i-90)))
    best = -1
    k=0
    for i in parameters:
        linear.C = i
        f1_measure = f1_score(yelp_valid_y,classifier.predict(valid_y_b), average='micro')
        if (best < f1_measure):
            k = i
            best = f1_measure
    linear.C = k   
    svm_test = linear.predict(test_y_b)
    test = f1_score(yelp_test_y,svm_test, average='micro')
    print("Test Result - svm classifier:")
    print("The F-measure is " + str(test) + " with C:" + str(linear.C) ) 

In [ ]:
performance(train_i_b,imdb_train_y,valid_i_b,imdb_valid_y,test_i_b,imdb_test_y)

In [ ]:
performance(train_i_f,imdb_train_y,valid_i_f,imdb_valid_y,test_i_f,imdb_test_y)